In [68]:
%load_ext autoreload
%autoreload 2
%matplotlib widget


import pyspedas
from pyspedas import tplot, options, get_data
from data_class import LoadArtemis, plot_electron_spectrogram

import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from Training.data_loader import MagnetotailDataset
from Helper.silence import silence_all
from tqdm import tqdm
import numpy as np

# Create the CDAWeb interface object
cdaweb_obj = pyspedas.CDAWeb()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import pandas as pd

# Read the Excel file and extract the first sheet with last 4 columns
df = pd.read_excel('ART_Xing_times_pos.xlsx', sheet_name=0)
df = df.iloc[:, -7:-1]  # Keep only the last 4 columns (last one is bugged so it is written with -5 to -1)

In [71]:
# Extracting first and last entries and first and last exits from the DataFrame
first_entries = df[df['first_entry'] == True]
last_entries  = df[df['last_entry']  == True]
first_exits   = df[df['first_exit']  == True]
last_exits    = df[df['last_exit']   == True]


# Convert the 'coupled time' column to a list for both last entries and first exits
first_entries_times = first_entries['coupled time'].tolist()
last_entries_times = last_entries['coupled time'].tolist()
first_exits_times = first_exits['coupled time'].tolist()
last_exits_times = last_exits['coupled time'].tolist()


In [72]:
### Magnetotail time processing ###

# Increment each entry in last_entries_times by 30 minutes
last_entries_times = [time + pd.Timedelta(minutes=30) for time in last_entries_times]

# Decrease each entry in first_exits_times by 30 minutes
first_exits_times = [time - pd.Timedelta(minutes=30) for time in first_exits_times]

In [73]:
### Magnetosheath time processing ###

# Create a list of times 1 day before each entry in first_exits_times
first_entries_times_start = [time - pd.Timedelta(days=1) for time in first_entries_times]
first_entries_times_end = [time - pd.Timedelta(minutes=90) for time in first_entries_times]  # Use the original times for the end

# Convert the times to strings for both last entries and first exits
first_entries_times_start_str = [str(time) for time in first_entries_times_start]
first_entries_times_end_str = [str(time) for time in first_entries_times_end]

# Create aentries of times 1 day after each entry in last_entries_times
last_exits_times_start = [time + pd.Timedelta(minutes=90) for time in last_exits_times]  # Use the original times for the start
last_exits_times_end = [time + pd.Timedelta(days=1) for time in last_exits_times]

# Convert the times to strings for both last entries and first exits
last_exits_times_start_str = [str(time) for time in last_exits_times_start]
last_exits_times_end_str = [str(time) for time in last_exits_times_end]



In [74]:
# Convert the lists of times to strings
last_entries_times_str = [str(time) for time in last_entries_times]
first_exits_times_str = [str(time) for time in first_exits_times]

# Check if the lengths of the two lists are equal
if len(last_entries_times_str) == len(first_exits_times_str):
    print("The lists have the same length.")
else:
    print("The lists have different lengths. This will cause issues in the next step.")

The lists have the same length.


In [77]:
# Add all magnetosheath data to the HDF5 file

# Add magnetosheath parts from before magnetotail entries
for i in tqdm(range(len(first_entries_times_start_str))):

    with silence_all():
        curr_data_class = LoadArtemis(start_time = first_entries_times_start_str[i], 
                                      end_time   = first_entries_times_end_str[i],
                                      satellite_list = ['THC'] )

        curr_data_class.convert_min_delta_all(start_time        = first_entries_times_start_str[i],
                                                end_time          = first_entries_times_end_str[i],
                                                dt_method         ='max', 
                                                custom_time_delta = 600)

        curr_data_class.extract_to_HDF5('training_data/artemis_P1_v4.h5')


 84%|████████▍ | 21/25 [04:19<00:49, 12.36s/it]


ValueError: Not enough valid points to interpolate for index 0

In [ ]:

# Add magnetosheath parts from after magnetotail exits
for i in tqdm(range(len(last_exits_times_start_str))):

    with silence_all():
        curr_data_class = LoadArtemis(start_time = last_exits_times_start_str[i], 
                                    end_time   = last_exits_times_end_str[i],
                                    satellite_list = ['THC'])

        curr_data_class.convert_min_delta_all(start_time        = last_exits_times_start_str[i],
                                            end_time          = last_exits_times_end_str[i],
                                            dt_method         ='max', 
                                            custom_time_delta = 600)

        curr_data_class.extract_to_HDF5('training_data/artemis_P1_v4.h5')

100%|██████████| 25/25 [33:22<00:00, 80.09s/it] 


In [ ]:
# Add all magnetotail data to the HDF5 file
for i in tqdm(range(len(first_entries_times_start_str))):

    with silence_all():
        curr_data_class = LoadArtemis(start_time = first_entries_times_start_str[i], 
                                      end_time   = last_exits_times_end_str[i]
                                      satellite_list = ['THC'])

        # curr_data_class.plot_pyspedas('B_ion_eflux')

        curr_data_class.convert_min_delta_all(start_time          = first_entries_times_start_str[i],
                                                end_time          = last_exits_times_end_str[i],
                                                dt_method         ='max', 
                                                custom_time_delta = 600)

        curr_data_class.extract_to_HDF5('testing_data/artemis_P1_v4.h5')



  0%|          | 0/25 [00:00<?, ?it/s]


KeyboardInterrupt: 